# Datalabframework

The datalabframework is a productivity framework for ETL, ML application. Simplifying some of the common activities which are typical in Data pipeline such as project scaffolding, data ingesting, start schema generation, forecasting etc.

In [1]:
import datalabframework as dlf

## Loading and Saving Data

In [2]:
dlf.project.load()

created SparkEngine
Init engine "spark"
Configuring packages:
  -  com.microsoft.sqlserver:mssql-jdbc:6.4.0.jre8
  -  com.oracle.jdbc:ojdbc8:12.2.0.1
  -  mysql:mysql-connector-java:8.0.12
  -  org.apache.hadoop:hadoop-aws:3.1.1
  -  org.postgresql:postgresql:42.2.5
  -  ru.yandex.clickhouse:clickhouse-jdbc:0.1.54
Configuring repositories:
  -  http://maven.icm.edu.pl/artifactory/repo/
  -  https://maven.xwiki.org/externals
Configuring conf:
  -  spark.hadoop.fs.s3a.access.key : ****** (redacted)
  -  spark.hadoop.fs.s3a.endpoint : http://minio:9000
  -  spark.hadoop.fs.s3a.impl : org.apache.hadoop.fs.s3a.S3AFileSystem
  -  spark.hadoop.fs.s3a.path.style.access : true
  -  spark.hadoop.fs.s3a.secret.key : ****** (redacted)
Connecting to spark master: local[1]
Engine context spark:2.4.1 successfully started


In [3]:
def equal(a,b):
    cnt = a.exceptAll(b).count() + b.exceptAll(a).count()
    return cnt==0

### Local files

The following show some load/save round trip on the local file system using various formats

In [4]:
#read data
df = dlf.engine().load('data/examples/sample.csv')
df.show()

+---+---+----+
|  a|  b|   c|
+---+---+----+
|yes|  1|1.41|
| no|  0|3.14|
+---+---+----+



In [5]:
# save in various format
dlf.save(df, 'data/save/foo.csv')
dlf.save(df, 'data/save/foo.json')
dlf.save(df, 'data/save/foo.parquet')

# with various compression
dlf.save(df, 'data/save/foo.json.bz2')
dlf.save(df, 'data/save/foo.csv.gz')

True

In [6]:
# using format specific save

dlf.save(df, 'data/save/bar.1', format='csv')
dlf.save(df, 'data/save/bar.2', format='json')
dlf.save(df, 'data/save/bar.3', format='parquet')

# with various compression
dlf.save(df, 'data/save/bar.4', format='csv', compression='bzip2')
dlf.save(df, 'data/save/bar.5', format='json', compression='gzip')
dlf.save(df, 'data/save/bar.6', format='parquet', compression='gzip')
dlf.save(df, 'data/save/bar.7', format='parquet', compression='snappy')

True

In [7]:
# round trip reading
df_tst = dlf.load('data/save/foo.csv')
assert(equal(df,df_tst))

df_tst = dlf.load('data/save/foo.json')
assert(equal(df,df_tst))

df_tst = dlf.load('data/save/foo.parquet')
assert(equal(df,df_tst))

df_tst = dlf.load('data/save/foo.json.bz2')
assert(equal(df,df_tst))

df_tst = dlf.load('data/save/foo.csv.gz')
assert(equal(df,df_tst))


In [8]:
# round trip reading (format specific load)
df_tst = dlf.load('data/save/bar.1', format='csv')
assert(equal(df,df_tst))

df_tst = dlf.load('data/save/bar.2', format='json')
assert(equal(df,df_tst))

df_tst = dlf.load('data/save/bar.3', format='parquet')
assert(equal(df,df_tst))

# df_tst = dlf.load('data/save/bar.4', format='csv', compression='bzip2')
# assert(equal(df,df_tst))

# df_tst = dlf.load('data/save/bar.5', format='json', compression='gzip')
# assert(equal(df,df_tst))

df_tst = dlf.load('data/save/bar.6', format='parquet', compression='gzip')
assert(equal(df,df_tst))

df_tst = dlf.load('data/save/bar.7', format='parquet', compression='snappy')
assert(equal(df,df_tst))


### Access data from HDFS

We can override the default resource provider, 
by explicitely passing a different provider

In [9]:
dlf.save(df, 'data/examples/bar.csv', 'hdfs')
dlf.save(df, 'data/examples/bar.json', 'hdfs')
dlf.save(df, 'data/examples/bar.parquet', 'hdfs')

True

In [10]:
df_tst = dlf.load('data/examples/bar.csv', 'hdfs')
assert(equal(df,df_tst))

df_tst = dlf.load('data/examples/bar.json', 'hdfs')
assert(equal(df,df_tst))

df_tst = dlf.load('data/examples/bar.parquet', 'hdfs')
assert(equal(df,df_tst))

### Access data from Minio

We can override the default resource provider, 
by explicitely passing a different provider

In [11]:
dlf.save(df, 'data/examples/bar.csv', 'minio')
dlf.save(df, 'data/examples/bar.json', 'minio')
dlf.save(df, 'data/examples/bar.parquet', 'minio')

True

In [12]:
df_tst = dlf.load('data/examples/bar.csv', 'minio')
assert(equal(df,df_tst))

df_tst = dlf.load('data/examples/bar.json', 'minio')
assert(equal(df,df_tst))

df_tst = dlf.load('data/examples/bar.parquet', 'minio')
assert(equal(df,df_tst))

### Access data from Web (HTTP/HTTPS)

Files from web will be first downloaded locally on the driver, then passed to spark ...

In [13]:
df_tst = dlf.load('https://raw.githubusercontent.com/natbusa/dlf-tutorial/master/data/examples/sample.csv')
assert(equal(df,df_tst))

Downloaded 27 bytes


### Access data from a Database (via jdbc connection)

#### MySQL: Sakila DB

In [14]:
query = """
    SELECT c.last_name,
        COUNT(p.amount) AS amount
    FROM customer c
    LEFT JOIN payment p
        ON c.customer_id = p.customer_id
    WHERE c.last_name like 'A%'
    GROUP BY  c.last_name
    ORDER BY  c.last_name ASC
    LIMIT 10;
"""
dlf.load(query, 'sakila').show()

+---------+------+
|last_name|amount|
+---------+------+
|    ABNEY|    21|
|     ADAM|    28|
|    ADAMS|    27|
|ALEXANDER|    27|
|   ALLARD|    32|
|    ALLEN|    31|
|  ALVAREZ|    27|
| ANDERSON|    24|
|   ANDREW|    25|
|  ANDREWS|    23|
+---------+------+



In [15]:
dlf.load('customer', 'sakila').show(5)

+-----------+--------+----------+---------+--------------------+----------+------+-------------------+-------------------+
|customer_id|store_id|first_name|last_name|               email|address_id|active|        create_date|        last_update|
+-----------+--------+----------+---------+--------------------+----------+------+-------------------+-------------------+
|          1|       1|      MARY|    SMITH|MARY.SMITH@sakila...|         5|  true|2006-02-14 22:04:36|2006-02-15 04:57:20|
|          2|       1|  PATRICIA|  JOHNSON|PATRICIA.JOHNSON@...|         6|  true|2006-02-14 22:04:36|2006-02-15 04:57:20|
|          3|       1|     LINDA| WILLIAMS|LINDA.WILLIAMS@sa...|         7|  true|2006-02-14 22:04:36|2006-02-15 04:57:20|
|          4|       2|   BARBARA|    JONES|BARBARA.JONES@sak...|         8|  true|2006-02-14 22:04:36|2006-02-15 04:57:20|
|          5|       1| ELIZABETH|    BROWN|ELIZABETH.BROWN@s...|         9|  true|2006-02-14 22:04:36|2006-02-15 04:57:20|
+-----------+---

In [16]:
df = dlf.load('customer', 'sakila')
df.select('customer_id', 'store_id', 'first_name', 'last_name').show(10)

+-----------+--------+----------+---------+
|customer_id|store_id|first_name|last_name|
+-----------+--------+----------+---------+
|          1|       1|      MARY|    SMITH|
|          2|       1|  PATRICIA|  JOHNSON|
|          3|       1|     LINDA| WILLIAMS|
|          4|       2|   BARBARA|    JONES|
|          5|       1| ELIZABETH|    BROWN|
|          6|       2|  JENNIFER|    DAVIS|
|          7|       1|     MARIA|   MILLER|
|          8|       2|     SUSAN|   WILSON|
|          9|       2|  MARGARET|    MOORE|
|         10|       1|   DOROTHY|   TAYLOR|
+-----------+--------+----------+---------+
only showing top 10 rows



#### Clickhouse: Taxes DB

In [17]:
query = """
    SELECT 
        SUM(emv) as total_emv, 
        SUM(tbea) as total_tbea, 
        SUM(bav) as total_bav, 
        tax_class,
        tax_rate 
    FROM tax_bills_nyc 
    GROUP BY tax_class, tax_rate
"""
dlf.load(query, 'taxes').show()

+---------------+-------------+---------------+--------------------+--------+
|      total_emv|   total_tbea|      total_bav|           tax_class|tax_rate|
+---------------+-------------+---------------+--------------------+--------+
|      1.31511E8|    3015446.0|     2.345737E7|2a - 4-6 unit res...|12.8550%|
|    2.3639859E8|    7038237.0|    5.4750993E7| 2c - co-op or co...|12.8550%|
|        7.778E7|    2107258.0|    1.6392523E7|2b - 7-10 unit re...|12.8550%|
|2.5456025934E10|1.119343167E9|1.0476817274E10|4 - commercial pr...|10.6840%|
|      1442000.0|      72190.0|       648900.0|3 - utility property|11.1250%|
|            0.0|    1574279.0|      1574279.0|                    |        |
|  7.279443191E9| 3.69685076E8|  2.875808079E9| 2 - residential,...|12.8550%|
|      1.31918E8|     979489.0|      5112968.0| 1 - small home, ...|19.1570%|
+---------------+-------------+---------------+--------------------+--------+



In [18]:
dlf.load('tax_bills_nyc', 'taxes').show(5)

+----------+--------------------+--------------------+--------------------+--------+---------+--------+--------+-----+------------+-----------+-----+
|       bbl|          owner_name|             address|           tax_class|tax_rate|      emv|    tbea|     bav|  tba|property_tax|condonumber|condo|
+----------+--------------------+--------------------+--------------------+--------+---------+--------+--------+-----+------------+-----------+-----+
|1001200009|DEPT OF TRANSPORT...|DEPT OF TRANSPORT...|3 - utility property|11.1250%|1442000.0| 72190.0|648900.0|     |           0|           |     |
|1000791001|138 FULTON CREW, ...|138 FULTON CREW L...| 2c - co-op or co...|12.8550%|2441000.0| 49588.0|385746.0|49588|       49588|       1333| unit|
|1000111001|15 STONE STREET C...|15 STONE STREET C...| 2c - co-op or co...|12.8550%|2373000.0| 41812.0|325260.0|41812|       41812|       2099| unit|
|1000111002|15 STONE STREET C...|15 STONE STREET C...| 2c - co-op or co...|12.8550%|3242000.0| 62718

#### MSSql: School DB

In [19]:
dlf.load('course', 'school').show(5)

+--------+-----------+-------+------------+
|CourseID|      Title|Credits|DepartmentID|
+--------+-----------+-------+------------+
|    1045|   Calculus|      4|           7|
|    1050|  Chemistry|      4|           1|
|    1061|    Physics|      4|           1|
|    2021|Composition|      3|           2|
|    2030|     Poetry|      2|           2|
+--------+-----------+-------+------------+
only showing top 5 rows



#### Oracle: HR DB

In [3]:
dlf.load('employees', 'human_resources').show(5)

+-----------+----------+---------+--------+------------+-------------------+-------+--------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|          HIRE_DATE| JOB_ID|  SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+-------------------+-------+--------+--------------+----------+-------------+
|        100|    Steven|     King|   SKING|515.123.4567|2003-06-17 00:00:00|AD_PRES|24000.00|          null|      null|           90|
|        101|     Neena|  Kochhar|NKOCHHAR|515.123.4568|2005-09-21 00:00:00|  AD_VP|17000.00|          null|       100|           90|
|        102|       Lex|  De Haan| LDEHAAN|515.123.4569|2001-01-13 00:00:00|  AD_VP|17000.00|          null|       100|           90|
|        103| Alexander|   Hunold| AHUNOLD|590.423.4567|2006-01-03 00:00:00|IT_PROG| 9000.00|          null|       102|           60|
|        104|     Bruce|    Ernst|  BERNST|590.423.4568|2007-0

#### Postgres: Pagila DB

In [4]:
# query from resource
md = dlf.Resource('select CAST(sum(amount) as DECIMAL(8,2)), customer_id from payment group by customer_id', 'pagila')
dlf.load(md).show(5)

+------+-----------+
|   sum|customer_id|
+------+-----------+
| 90.77|        184|
|145.70|         87|
|109.78|        477|
|157.65|        273|
|159.68|        550|
+------+-----------+
only showing top 5 rows



In [5]:
#Use JOIN to display the total amount rung up by each staff member
# use tables 'staff' and 'payment'

query = """
    SELECT 
        CAST(SUM(p.amount) AS DECIMAL(16,2)) as total_sales, 
        s.last_name, 
        s.first_name
    FROM payment p 
    INNER JOIN staff s ON p.staff_id = s.staff_id 
    GROUP BY s.last_name, s.first_name
    """
df = dlf.load(query, 'pagila')
df.show()

+-----------+---------+----------+
|total_sales|last_name|first_name|
+-----------+---------+----------+
|   33489.47|  Hillyer|      Mike|
|   33927.04| Stephens|       Jon|
+-----------+---------+----------+



In [6]:
md = dlf.Resource('total_sales', 'pagila')
dlf.save(df, md)

True

In [7]:
# round trip read back
df = dlf.load('total_sales', 'pagila')
df.show()

+-----------+---------+----------+
|total_sales|last_name|first_name|
+-----------+---------+----------+
|   33489.47|  Hillyer|      Mike|
|   33927.04| Stephens|       Jon|
+-----------+---------+----------+



In [8]:
# check if the pyspark DataFrame class is monkey patched
df.datalabframework()

datalabframework 0.8.2


In [9]:
df.data.collect(1)

total_sales last_name first_name
0    33489.47   Hillyer       Mike

In [10]:
# generate a new dataframe from the original one, 
# by providing new data and retaining the original schema

from decimal import Decimal as d
df = df.rows.overwrite([(d(12345.67),'Dereck', 'Eve')])
df.show()

+-----------+---------+----------+
|total_sales|last_name|first_name|
+-----------+---------+----------+
|   12345.67|   Dereck|       Eve|
+-----------+---------+----------+



In [11]:
# append new records and reload
dlf.save(df, 'total_sales', 'pagila', mode='append')
dlf.load('total_sales', 'pagila').show()

+-----------+---------+----------+
|total_sales|last_name|first_name|
+-----------+---------+----------+
|   33489.47|  Hillyer|      Mike|
|   33927.04| Stephens|       Jon|
|   12345.67|   Dereck|       Eve|
+-----------+---------+----------+



### From one provider to the other

In [12]:
dlf.load('payment', 'pagila').save('data/payment', 'hdfs')

True